In [1]:
# Import necessary libraries
from surprise import Dataset, Reader, KNNBasic
import pandas as pd

In [2]:
# Read Data from Food_Data, Food_Rating, and Food_Tags
food_data = pd.read_csv("ORS Data/Food_Data.csv")

food_rating = pd.read_csv('ORS Data/Food_Rating.csv')

food_tags = pd.read_csv("ORS Data/food_tags.csv")

In [8]:
# Merge dataframes to create a unified dataset
merged_data = pd.merge(food_data, food_rating, on='FoodID')
merged_data = pd.merge(merged_data, food_tags, on=['FoodID', 'UserID'])

In [9]:
merged_data

,FoodID,FoodName,Cuisine,UserID,Rating,Tags
0,1,Masala Karela Recipe,Indian,16,4,Exquisite
1,1,Masala Karela Recipe,Indian,16,4,Mediocre
2,1,Masala Karela Recipe,Indian,16,2,Exquisite
3,1,Masala Karela Recipe,Indian,16,2,Mediocre
4,1,Masala Karela Recipe,Indian,1,1,Tasteless
...,...,...,...,...,...,...
1889,57,One Pot Methi Mushroom Brown Rice Pulao Recipe,Indian,4,5,Masterfully prepared
1890,57,One Pot Methi Mushroom Brown Rice Pulao Recipe,Indian,4,5,Sumptuous
1891,57,One Pot Methi Mushroom Brown Rice Pulao Recipe,Indian,4,5,Masterfully prepared
1892,57,One Pot Methi Mushroom Brown Rice Pulao Recipe,Indian,4,5,Sumptuous


In [10]:
# Define a reader to parse the data
reader = Reader(rating_scale=(1, 5))

In [11]:
# Load data into Surprise dataset
data = Dataset.load_from_df(merged_data[['UserID', 'FoodID', 'Rating']], reader)


In [12]:
data

In [13]:
# Build a collaborative filtering model (k-NN algorithm)
sim_options = {
    'name': 'cosine',  # Use cosine similarity
    'user_based': False  # Item-based collaborative filtering
}
model = KNNBasic(sim_options=sim_options)


In [14]:
# Train the model
trainset = data.build_full_trainset()
model.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [18]:
# Function to get recommendations for a given food item
def get_recommendations(food_name):
    # Find the FoodID for the given food_name
    food_id = food_data[food_data['FoodName'] == food_name]['FoodID'].values[0]

    # Get the inner ID for the food in the model
    food_inner_id = model.trainset.to_inner_iid(food_id)

    # Get the 5 nearest neighbors for the given food
    food_neighbors = model.get_neighbors(food_inner_id, k=5)

    # Convert inner IDs back to FoodIDs
    food_neighbors = [model.trainset.to_raw_iid(inner_id) for inner_id in food_neighbors]

    # Get the names of the recommended foods
    recommended_foods = food_data[food_data['FoodID'].isin(food_neighbors)]['FoodName'].tolist()

    return recommended_foods

In [19]:
# Example: Get recommendations for Food Item
food_item = input("Please enter a food item to get recommendations")
recommendations = get_recommendations(food_item)
print("ORS running to fetch results..")

print(f"Recommended foods for {food_item} : {recommendations}")


ORS running to fetch results..
Recommended foods for Masala Karela Recipe : ['Spicy Chilli Garlic Noodles Recipe', 'Mustard Vegetable Curry Recipe', 'Aloo Matar Masala Khichdi Recipe', 'Mushroom Dumpling Recipe With Potatoes (Momos)', 'Till Wale Aloo Recipe - Sesame Potato Sabzi']
